In [39]:
import numpy
import urllib
import geojson
import pandas as pd
import requests
import json
import arcpy
import requests
from shapely.geometry import shape, Polygon, MultiPolygon
import numpy as np
import arcpy
import psycopg2
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

# https://towardsdatascience.com/json-and-apis-with-python-fba329ef6ef0

In [5]:
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Planning_Primary_Zoning/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'
response = requests.get(url)
data = response.json()

zoningCode = pd.json_normalize(data['features']) # get into pandas

# create variable that holds all features that are not polygon or multipolygon
non_polygons = zoningCode[(zoningCode['geometry.type'] != 'Polygon') & (zoningCode['geometry.type'] != 'MultiPolygon')]

# create error code column and put in null if feature is null
zoningCode.loc[zoningCode['properties.ZONE_CODE'].isnull(), 'ERROR_CODE'] = 'null'
zoningCode.loc[zoningCode['properties.Shape__Area'].isnull(), 'ERROR_CODE'] = 'null'
zoningCode.loc[zoningCode['properties.OBJECTID'].isnull(), 'ERROR_CODE'] = 'null'
zoningCode.loc[zoningCode['geometry.coordinates'].isnull(), 'ERROR_CODE'] = 'null'
# create error code column and put in wrongtype if feature is not polygon or multipolygon
zoningCode.loc[non_polygons.index, 'ERROR_CODE'] = 'wrongType'

# Convert geom to WKT format
def coords_to_wkt(coords):
    if isinstance(coords[0][0], list):
        # if multiPolygon
        poly_list = [geojson.Polygon(poly) for poly in coords]
        multi_poly = geojson.MultiPolygon(poly_list)
        return geojson.dumps(multi_poly).replace('[', '(').replace(']', ')')
    else:
        # if polygon
        poly = geojson.Polygon(coords)
        return geojson.dumps(poly).replace('[', '(').replace(']', ')')

zoningCode['geometry'] = zoningCode['geometry.coordinates'].apply(coords_to_wkt)

# rename columns so it matches tables names in postgis
zoningCode = zoningCode.rename(columns={
    'properties.OBJECTID': 'ZONE_ID', 
    'geometry.type': 'ZONE_TYPE', 
    'properties.ZONE_CODE': 'ZONE_CODE', 
    'properties.Shape__Area': 'AREA'    
})

# list certain columns that I want
zoningCode = zoningCode.loc[:, ['ZONE_ID', 'ZONE_TYPE', 'ZONE_CODE', 'geometry', 'AREA', 'ERROR_CODE']]

print(zoningCode)

# insert into, put a break into it

      ZONE_ID ZONE_TYPE ZONE_CODE  \
0           1   Polygon       R2B   
1           2   Polygon        C1   
2           3   Polygon        I1   
3           4   Polygon        C4   
4           5   Polygon       R2B   
...       ...       ...       ...   
2210     2211   Polygon       C3A   
2211     2212   Polygon       C3A   
2212     2213   Polygon        C1   
2213     2214   Polygon        I2   
2214     2215   Polygon        I1   

                                               geometry           AREA  \
0     {"type": "MultiPolygon", "coordinates": (((-93...    2189.808594   
1     {"type": "MultiPolygon", "coordinates": (((-93...    2171.085938   
2     {"type": "MultiPolygon", "coordinates": (((-93...   84326.816406   
3     {"type": "MultiPolygon", "coordinates": (((-93...   12925.625000   
4     {"type": "MultiPolygon", "coordinates": (((-93...   28257.261719   
...                                                 ...            ...   
2210  {"type": "MultiPolygon", "coord

In [69]:
import requests
import psycopg2
import json

# Make a GET request to the URL and parse the response as JSON
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Planning_Primary_Zoning/FeatureServer/0/query?outFields=ZONE_CODE&where=1%3D1&f=geojson'
response = requests.get(url)
data = json.loads(response.content)

features = data['features']
    
conn = psycopg2.connect(database="lab0", user="postgres", password="Homework13", host="34.27.240.213")
cur = conn.cursor()

for feature in features:
    zone_code = feature['properties']['ZONE_CODE']
    geometry = json.dumps(feature['geometry'])
    cur.execute("SELECT ST_AsText(ST_GeomFromGeoJSON(%s))", (geometry,))
    wkt_geometry = cur.fetchone()[0]

    # Check for null values in zone_code and wkt_geometry
    error_code = None
    if zone_code is None:
        error_code = 'ZoneError'
    elif wkt_geometry is None:
        error_code = 'GeomError'
    
    cur.execute("INSERT INTO MPLSZoning (zone_code, geometry, error_code) VALUES (%s, %s, %s)", (zone_code, wkt_geometry, error_code))
    conn.commit()

cur.close()
conn.close()


KeyboardInterrupt: 

In [ ]:
CREATE TABLE MPLSZoning
(
    ZONE_ID integer,
    ZONE_TYPE text,
    ZONE_CODE text, 
    AREA double precision,
    ERROR_CODE text,
    geometry geometry
);

In [76]:
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Planning_Primary_Zoning/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

response = requests.get(url)
zoningData = response.json() # pulls out JSON data

zoningDataDF = pd.json_normalize(zoningData['features']) # normalize data so it can be read in pandas dataframe
print(zoningDataDF.head())


      type  id geometry.type  \
0  Feature   1       Polygon   
1  Feature   2       Polygon   
2  Feature   3       Polygon   
3  Feature   4       Polygon   
4  Feature   5       Polygon   

                                geometry.coordinates  properties.OBJECTID  \
0  [[[-93.3037816866939, 45.0077970445671], [-93....                    1   
1  [[[-93.3088637930648, 45.0096321731455], [-93....                    2   
2  [[[-93.2401697007511, 44.9582021397118], [-93....                    3   
3  [[[-93.2556792476983, 44.9490392740053], [-93....                    4   
4  [[[-93.2323502580779, 44.900843000181], [-93.2...                    5   

  properties.ZONE_CODE  properties.RuleID  properties.Shape__Area  \
0                  R2B                4.0             2189.808594   
1                   C1               12.0             2171.085938   
2                   I1               24.0            84326.816406   
3                   C4               16.0            12925.625000   

## call data and put it into pandas dataframe

## Finding data that does not have polygon type

In [8]:
# after robs numpy lecture, lets get this into a array so I can use numpy!

all_coordinates = []

# Loop through polygons and get coordinates
for polygon in zoningData['features']:
    coordinates = polygon['geometry']['coordinates']
    all_coordinates.append(coordinates)

# Convert coordinates to a numpy array
numpy_array = np.array(all_coordinates)

print(numpy_array)

[list([[[-93.3037816866939, 45.0077970445671], [-93.3044171761646, 45.0077988081563], [-93.3044188303179, 45.0079973020938], [-93.304310786979, 45.0079967711909], [-93.3038133441888, 45.0079926935659], [-93.3037866283911, 45.0079931545513], [-93.3037816866939, 45.0077970445671]]])
 list([[[-93.3088637930648, 45.0096321731455], [-93.308221316948, 45.0096294300474], [-93.3082203065919, 45.0094377858855], [-93.3083470140401, 45.0094382272662], [-93.3088405744317, 45.0094382623233], [-93.308864048216, 45.0094382002494], [-93.3088637930648, 45.0096321731455]]])
 list([[[-93.2401697007511, 44.9582021397118], [-93.2401933013425, 44.9582023429819], [-93.2407854571805, 44.9582077281548], [-93.2407919777382, 44.9582076265245], [-93.241118483325, 44.9582025622282], [-93.2412880866091, 44.9582033547577], [-93.240713780009, 44.9573702470881], [-93.2406807557925, 44.9573134777399], [-93.2408511418181, 44.9573133891943], [-93.2412922835654, 44.9573140876631], [-93.2414717471595, 44.9572677605277], [-

<ipython-input-8-05aee11832a8>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_array = np.array(all_coordinates)


In [9]:
# count the number of polygons
num_polygons = len(zoningData['features'])
print("Number of features:", num_polygons)

Number of features: 2215


## Check null values

In [12]:
# Check for null values

nullZoneCodes = zoningDataDF[zoningDataDF['properties.ZONE_CODE'].isnull()]

print(nullZoneCodes)

         type    id geometry.type  \
2110  Feature  2111       Polygon   

                                   geometry.coordinates  properties.OBJECTID  \
2110  [[[-93.2785782827594, 44.9681617941297], [-93....                 2111   

     properties.ZONE_CODE  properties.RuleID  properties.Shape__Area  \
2110                 None                NaN                   0.625   

      properties.Shape__Length  
2110                 56.975468  


## Check for duplicate records

In [ ]:
print(df.duplicated().sum())

## find overlapping polygons

## find polygons whose lines intersect

In [14]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'